In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import hyperopt
import sys
sys.path.append('../spock/')
from simsetup import get_sim
from modelfitting import ROC_curve, stable_unstable_hist, calibration_plot, unstable_error_fraction
from simsetup import get_sim
try:
    plt.style.use('paper')
except:
    pass
%matplotlib inline

/mnt/ssd/miniconda3/envs/yba/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
datapath = '../training_data/'
csvpath = '../csvs/'
dset = 'resonant/'
Norbits = 1e4
Nout = 80
featureargs = (Norbits, Nout) # params to pass feature function
# featurefolder = 'featuresNorbits{0:.1f}Nout{1}trio/'.format(Norbits, Nout)
featurefolder = 'oldfeatures/'.format(Norbits, Nout)
trainingdatafolder = datapath+dset+featurefolder
csvfolder = csvpath+dset
print(trainingdatafolder)
print(csvfolder)

../training_data/resonant/oldfeatures/
../csvs/resonant/


In [3]:
dataset = pd.read_csv(trainingdatafolder+"trainingdata.csv", index_col = 0)
labels = pd.read_csv(trainingdatafolder+"labels.csv", index_col=0)

In [ ]:
# intial condition folder here
icfolder = "/home/yba/spock/data/resonant/"

In [ ]:
labels_with_tmax = pd.read_csv("/home/yba/spock/training_data/resonant/labels_with_tmax.csv", index_col=0)

In [ ]:
## cheating here
dataset['tmax'] = labels_with_tmax['tmax']

In [ ]:
dataset

In [ ]:
def hasnull(row):
    numnulls = row.isnull().sum()
    if numnulls == 0:
        return 0
    else:
        return 1

def tmax(row):
    sim = get_sim(row, icfolder)
    mtotal = sim.particles[1].m + sim.particles[2].m  + sim.particles[3].m
    alpha13 = sim.particles[1].a / sim.particles[3].a 
    ec13 = 1-alpha13 
    Tsec = 4* sim.particles[0].m/mtotal * ec13 * ec13  * sim.particles[3].P
    Norbits = 5* Tsec
    if Norbits > 10**6:
        Norbits = 10**6
    return Norbits

In [ ]:
near = ['EMcrossnear', 'EMfracstdnear', 'EPstdnear', 'MMRstrengthnear']
far = ['EMcrossfar', 'EMfracstdfar', 'EPstdfar', 'MMRstrengthfar']
megno = ['MEGNO', 'MEGNOstd']

features = near + far + megno

add columns for tmax and hasnull manually

In [ ]:
%%time
if 'tmax' not in dataset.columns:
    dataset['hasnull'] = dataset.apply(hasnull, axis=1)
    # dataset['tmax'] = dataset.head(100).apply(tmax, axis=1)
    # dataset['tmax'] = dataset.apply(lambda x:1e4, axis=1) # this version would just set tmax=1e4 for all of them

    dataset.to_csv(trainingdatafolder+"trainingdata.csv", encoding='ascii')

Now we do the filtering manually. This is taking only systems with instability times > 1e4 AND no NaNs. Would adjust this for each case. 

We were worried that we were using filter=..., which was filtering out any rows that had any NaNs in them. We wanted to compare results when we don't include that filter, and only filtered for instability_time.

In [ ]:
mask = (labels['instability_time'] > dataset['tmax']) & (dataset['hasnull'] == 0)
#mask = (labels['instability_time'] > labels['tmax']) & (dataset['hasnull'] == 0)
y = labels[mask]['Stable']
X = dataset[mask][features]
tinst = labels[mask]['instability_time']

Now we do the train test split manually. We take 80\% of the rows for training, 20\% for testing (this is what we were doing before too)

In [ ]:
Nrows = int(0.8*X.shape[0])
trainX = X.iloc[:Nrows, :]
trainY = y.iloc[:Nrows]
testX = X.iloc[Nrows:, :]
testY = y.iloc[Nrows:]
test_tinst = tinst.iloc[Nrows:]

dtrain = xgb.DMatrix(trainX, trainY)
dtest = xgb.DMatrix(testX, testY)

In [ ]:
space ={'max_depth': hp.qloguniform('x_max_depth', np.log(5), np.log(20), 1),
        'min_child_weight': hp.loguniform('x_min_child', 0, np.log(20)),
        'subsample': hp.uniform ('x_subsample', 0.8, 1),
}

def objective(params):
    clf = XGBClassifier(n_jobs=16, n_estimators = 50,
                            max_depth = int(params['max_depth']), 
                            min_child_weight = params['min_child_weight'],
                            subsample = params['subsample'],
                            learning_rate = 0.15, seed = 0)
    
    score = xgb.cv(clf.get_xgb_params(), dtrain, nfold = 5, metrics = "auc", early_stopping_rounds=10)
    avg_score =  np.mean(score["test-auc-mean"])
    error = np.mean(score["test-auc-std"])
    
    print("SCORE:", avg_score, "ERROR", error)#, "HOLDOUT SCORE", test_score)
    return{'loss':1-avg_score, 'status': STATUS_OK, "cv_score":avg_score , "cv_error":error}


In [ ]:
trials = Trials()
import time
start = time.time()
best = fmin(fn=objective, space = space, algo = tpe.suggest, max_evals = 100, trials = trials, rstate=np.random.RandomState(seed=0))
end = time.time()
print("Optimization Time: %f seconds", (end  -start))

# max_depth controls depth of trees

12 lets the model use all the features and improvements seem minor beyond that

In [ ]:
depths = trials.vals['x_max_depth']
min_childs = trials.vals['x_min_child']
aucs = np.array([1-x['loss'] for x in trials.results])

fig, ax = plt.subplots()
ax.plot(depths, aucs, '.')
ax.set_xlabel('max_depth')
ax.set_ylabel('CV AUC')

# min child weight acts as a regularizer, penalizing complex models. Larger min_child_weight = larger penalization

In [ ]:
fig, ax = plt.subplots()
ax.plot(min_childs, aucs, '.')
ax.set_xlabel('min_child_weight')
ax.set_ylabel('CV AUC')

In [ ]:
fig, ax = plt.subplots()
cb = ax.scatter(depths, min_childs, c=aucs-0.93)
plt.colorbar(cb, label='CV AUC - 0.93')
ax.set_xlabel('max_depth')
ax.set_ylabel('min_child_weight')

# Marginal improvements beyond max_depth of 13, so choose the least complex model

In [ ]:
model = XGBClassifier(learning_rate = 0.03, 
                         max_depth = 20, 
                         subsample = 0.95,
                         min_child_weight = 10)

score = xgb.cv(model.get_xgb_params(), dtrain, nfold = 5, metrics = "auc", verbose_eval=True, num_boost_round=400)

# Going beyond ~100 trees does not improve CV, so cut off training there to avoid overfitting

In [ ]:
n_estimators = 100
fig, ax = plt.subplots()
ax.plot(score.index, score['train-auc-mean'], label='Train')
ax.plot(score.index, score['test-auc-mean'], label='Test')
ax.axvline(n_estimators, linestyle='--')
ax.legend()
ax.set_xlabel('n_estimators (num trees)')
ax.set_ylabel('CV AUC score')

In [ ]:
model.set_params(n_estimators = n_estimators)
model.fit(trainX, trainY)

In [ ]:
model.save_model(datapath+'../spock/models/spock_fixtmaxfeature_tmax.json')

# SPOCK

For each training data, need to load the corresponding dataset and model. Also need to change MASK

### Old version rebound - ic 10^4, filter 10^4 - origin paper

In [4]:
old_model = XGBClassifier()
old_model.load_model(datapath+'../spock/models/featureclassifier.json')

XGBoostError: [10:42:00] /croot/xgboost-split_1675119646044/work/include/xgboost/json.h:73: Invalid cast, from Integer to Boolean
Stack trace:
  [bt] (0) /mnt/ssd/miniconda3/envs/yba/lib/libxgboost.so(+0x981e4) [0x7fd897aaa1e4]
  [bt] (1) /mnt/ssd/miniconda3/envs/yba/lib/libxgboost.so(xgboost::JsonBoolean const* xgboost::Cast<xgboost::JsonBoolean const, xgboost::Value const>(xgboost::Value const*)+0x310) [0x7fd897ae84b0]
  [bt] (2) /mnt/ssd/miniconda3/envs/yba/lib/libxgboost.so(xgboost::RegTree::LoadModel(xgboost::Json const&)+0x10ba) [0x7fd897cd71da]
  [bt] (3) /mnt/ssd/miniconda3/envs/yba/lib/libxgboost.so(xgboost::gbm::GBTreeModel::LoadModel(xgboost::Json const&)+0x63d) [0x7fd897bb17cd]
  [bt] (4) /mnt/ssd/miniconda3/envs/yba/lib/libxgboost.so(xgboost::gbm::GBTree::LoadModel(xgboost::Json const&)+0x185) [0x7fd897b9a2d5]
  [bt] (5) /mnt/ssd/miniconda3/envs/yba/lib/libxgboost.so(+0x1bbc57) [0x7fd897bcdc57]
  [bt] (6) /mnt/ssd/miniconda3/envs/yba/lib/libxgboost.so(XGBoosterLoadModel+0x

In [ ]:
roc_auc, fpr, tpr, ROCthresholds = ROC_curve(old_model, testX, testY)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.0])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC curve (AUC = {0:.3f})'.format(roc_auc))

In [ ]:
fixtmax_model = XGBClassifier()
old_model.load_model(datapath+'../spock/models/spock_fixtmaxfeature_tmax.json')

### new rebound - integrate to tmax

In [ ]:
vary_tmax_model = XGBClassifier()
vary_tmax_model.load_model(datapath+'../spock/models/featureclassifier.json')

In [ ]:
roc_auc, fpr, tpr, ROCthresholds = ROC_curve(vary_tmax_model, testX, testY)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.0])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC curve (AUC = {0:.3f})'.format(roc_auc))

In [ ]:
new_fix_tmax_model = XGBClassifier()
new_fix_tmax_model.load_model(datapath+'../spock/models/spock_fixtmaxfeature_tmax.json')

In [ ]:
roc_auc, fpr, tpr, ROCthresholds = ROC_curve(new_fix_tmax_model, testX, testY)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.0])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC curve (AUC = {0:.3f})'.format(roc_auc))

In [ ]:
stride = 10
np.savetxt('../spock/models/resROC.txt', (ROCthresholds[::stride], tpr[::stride], fpr[::stride]))

In [ ]:
ROCthresholds, tpr, fpr = np.loadtxt('../spock/models/resROC.txt')
for i in range(0,len(tpr), 15):
    print("Threshold {0}, TPR = {1}, FPR = {2}".format(ROCthresholds[i], tpr[i], fpr[i]))

In [ ]:
# feature importances
feat_imp = pd.Series(model.get_booster().get_fscore()).sort_values(ascending=False)
ax = feat_imp.plot.barh(figsize=(12,8), fontsize=24)
ax.set_xlabel('Feature Importance Score', fontsize=24)
ax.invert_yaxis()
plt.savefig('featureimportances.pdf', bbox_inches='tight')

In [ ]:
feat_imp

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, confusion_matrix, auc
from sklearn import metrics

In [ ]:
# Histogram:
bins = 50
Stable, Unstable = stable_unstable_hist(model, testX, testY)
print(Unstable)

fig, ax1 = plt.subplots()
n, bins, patches = ax1.hist(Unstable, bins, alpha=0.5, label='No', color='blue', edgecolor = "black")
ax1.set_xlabel('Predicted Probability', fontsize=14)
ax1.set_ylabel('Unstable',  fontsize=14, color='blue')
for tl in ax1.get_yticklabels():
    tl.set_color('blue')
    
ax2 = ax1.twinx()
n, bins , patches = ax2.hist(Stable, bins, alpha=0.5, label='Yes',color='green', edgecolor = "black")
ax2.set_ylabel('Stable', fontsize=14, color='green')
for tl in ax2.get_yticklabels():
    tl.set_color('green')
       
ax1.set_ylim([0,35*n[-1]]) # goes up to ~4300
ax2.set_ylim([0,1.1*n[-1]]) # goes up to ~2100
fig.savefig('stable_unstable_comparison.png')

In [ ]:
bincenters, fracstable, errorbars = calibration_plot(model, testX, testY, bins=8)
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(np.linspace(0,1,100), np.linspace(0,1,100), '--')
ax.errorbar(bincenters, fracstable, errorbars)
ax.set_xlim([0,1])
ax.set_ylim([0,1])
ax.set_xlabel('Predicted Probability of Stability')
ax.set_ylabel('Fraction actually stable')

In [ ]:
thresh = 0.34 # for 10% FPR
bincenters, errorfracs, errorbars = unstable_error_fraction(model, testX, testY, test_tinst, thresh, bins=10)
fig, ax = plt.subplots(figsize=(8,6))
ax.errorbar(bincenters, errorfracs, errorbars)
ax.set_ylim([0,1])
ax.set_xlabel('Log Instability Time')
ax.set_ylabel('Error Fraction')
ax.set_title('Fraction of unstable systems mislabeled as stable')